Ingeniería de Software Basada en la Nube

#  Unidad 5 - Laboratorio
__________________

## Objetivos

**1.** Despelgar un componente de software en un **clúster** de **Kubernetes**.

**2.** Consttuir dos componentes nuevos: **API Gateway** y **Web Front-End**.


## Parte 1: Clustering

Desarrollar el siguiente laboratorio de **Qwiklabs**: [Kubernetes Engine: Qwik Start](https://www.qwiklabs.com/focuses/878?locale=es&parent=catalog). Para ello tener en cuenta las siguientes consideraciones:

**1.** Leer muy bien la sección **Descripción general**, revisando en detalle cada uno de los conceptos allí presentados.

**2.** Omitir la sección **Configuración y requisitos**. Para el desarrollo del laboratorio, hacer uso de los recursos de [GCP](https://console.cloud.google.com/) asociados a cada cuenta personal.

**3.** En el clúster creado, desplegar el componente **isbn-users-ms** del Laboratorio de la Unidad 4.

**4.** Escalar horizontalmente el componente **isbn-users-ms**, siguiendo como referencia la siguiente guía de **Google Kubernetes Engine (GKE)**: [Scaling an application](https://cloud.google.com/kubernetes-engine/docs/how-to/scaling-apps).

## Parte 2: Nuevos Componentes

### Componente #1

* **Tipo de componente:** API Gateway
* **Nombre:** isbn-ag
* **Lenguaje de Programación:** Python
* **Framework:** Flask

**1.** Crear un directorio llamado **isbn-ag**.

**2.** Dentro del directorio, crear los siguientes directorios:

    - consumers/
    - services/
    - controllers/

**3.** En el directorio **consumers**, crear un directorio con nombre **users_ms** y dentro de él un archivo **user_consumer.py**:

In [ ]:
import requests

from config import ISBN_USERS_MS_API_URL

base_url = ISBN_USERS_MS_API_URL

# Create User
def create_user(data):

    url = base_url + '/api/user'

    response = requests.post(url, json=data)

    return response

**4.** En el directorio **consumers**, crear un directorio con nombre **tasks_ms** y dentro de él un archivo **task_consumer.py**:

In [ ]:
import requests

from config import ISBN_TASKS_MS_API_URL

base_url = ISBN_TASKS_MS_API_URL

# Create Task
def create_task(data):

    url = base_url

    response = requests.post(url, json=data)

    return response

**5.** En el directorio **services**, crear un archivo **record_service.py**:

*Nota:* en la línea 23 se asigna el valor **1** al atributo **user_id**. Sin embargo, este valor no corresponde al **user_id** correcto. Para asignar el **user_id** correcto, es necesario modificar el microservicio **isbn-users-ms** para que el método **POST** de creación de usuario retorne el **user_id** asignado por la base de datos, y de esta manera, el valor correcto pueda ser asignado al atributo.

In [ ]:
from consumers.users_ms.user_consumer import *
from consumers.tasks_ms.task_consumer import *

class RecordService:

    @staticmethod
    def create_record_service(data):

        user_data = {
            "name": data['user_name'],
            "email": data['user_email']
        }

        user = create_user(user_data)

        if user.status_code == 201:

            task_data = {
                "id": data['task_id'],
                "details":{
                    "name": data['task_name'],
                    "description": data['task_description'],
                    "user_id": "1"
                }
            }

            task = create_task(task_data)

            if task.status_code == 201:

                return True

        return False

**6.** En el directorio **controllers**, crear un archivo **record_controller.py**:

In [ ]:
from flask import Blueprint, request, jsonify
from services.record_service import RecordService

record_api = Blueprint('record_api', __name__)

@record_api.route('/api/record', methods=['POST'])
def create_record_controller():

    data = request.get_json()

    response = RecordService.create_record_service(data)

    if response == True:
        return jsonify("Record has been successfully created."), 201

    return jsonify("Error"), 400

**7.** En la raíz del proyecto (directorio **isbn-ag**), crear archivo **app.py**:

In [ ]:
from flask import Flask
from controllers.record_controller import *

app = Flask(__name__)

app.register_blueprint(record_api)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=4500)

**8.** En la raíz del proyecto (directorio **isbn-ag**), crear archivo **requirements.txt**:

In [ ]:
Flask==2.3.3
requests==2.31.0

**9.** En la raíz del proyecto (directorio **isbn-ag**), crear archivo **Dockerfile**:

In [ ]:
FROM python

WORKDIR /app

COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

COPY . .

EXPOSE 4500

CMD ["python", "app.py"]

**10.** En la raíz del proyecto (directorio **isbn-ag**), crear archivo **config.py**:

In [ ]:
import os

# Environment Variables (Microservices)

ISBN_USERS_MS_API_URL = os.environ.get('ISBN_USERS_MS_API_URL')
ISBN_TASKS_MS_API_URL = os.environ.get('ISBN_TASKS_MS_API_URL')

**11.** Crear imagen Docker para el componente **isbn-ag**:

In [ ]:
docker build -t isbn-ag .

**12.** Ejecutar el componente **isbn-ag** en un contenedor Docker:

*Nota:* **-e** representa la declaración de una variable de entorno en el contexto del contenedor.

*Importante:* reemplazar **X** por la URL asociada al microservicio **isbn-users-ms**, y **Y** por la URL asociada al microservicio **isbn-tasks-ms**.

In [ ]:
docker run -p 4500:4500 -e ISBN_USERS_MS_API_URL=X -e ISBN_TASKS_MS_API_URL=Y --name isbn-ag isbn-ag

**13.** Consumir la operación expuesta por el API Gateway usando un **Cliente HTTP**.

* **Método HTTP:** POST
* **URL:** http://localhost:4500/api/record
* **Cuerpo del mensaje (body):**

In [ ]:
{
    "user_name": "Jeisson Vergara",
    "user_email": "javergarav@unal.edu.co",
    "task_id": "abc-123",
    "task_name": "Tarea 1",
    "task_description": "Clase de ISBN 2024ii"
}

### Componente #2

* **Tipo de componente:** Web Front-End
* **Nombre:** isbn-wa
* **Lenguaje de Programación:** Python
* **Framework:** Flask

**1.** Crear un directorio llamado **isbn-wa**.

**2.** Dentro del directorio, crear los siguientes directorios:

    - consumers/
    - services/
    - controllers/
    - static/
    - templates/

**3.** En el directorio **consumers**, crear un directorio con nombre **ag** y dentro de él un archivo **record_consumer.py**:

In [ ]:
import requests

from config import ISBN_AG_API_URL

base_url = ISBN_AG_API_URL

# Create Record
def create_record(data):

    url = base_url + '/api/record'

    response = requests.post(url, json=data)

    return response

**4.** En el directorio **services**, crear archivo **record_service.py**:

In [ ]:
from consumers.ag.record_consumer import *

class RecordService:

    @staticmethod
    def create_record(user_name, user_email, task_id, task_name, task_description):

        data = {
            "user_name": user_name,
            "user_email": user_email,
            "task_id": task_id,
            "task_name": task_name,
            "task_description": task_description
        }

        return create_record(data)

**5.** En el directorio **controllers**, crear archivo **record_controller.py**:

In [ ]:
from flask import Blueprint, render_template, request, jsonify, redirect, url_for
from services.record_service import RecordService

record_api = Blueprint('record_api', __name__)

@record_api.route('/record', methods=['POST'])
def create_record():

    data = request.form
    user_name = data.get('user_name')
    user_email = data.get('user_email')
    task_id = data.get('task_id')
    task_name = data.get('task_name')
    task_description = data.get('task_description')

    if not user_name:
        return jsonify({'error': 'User name is required'}), 400

    if not task_name:
        return jsonify({'error': 'Task name is required'}), 400

    RecordService.create_record(user_name, user_email, task_id, task_name, task_description)
    return redirect(url_for('record_api.index'))

@record_api.route('/')
def index():
    return render_template('index.html')

**6.** En el directorio **templates**, crear archivo **index.html**:

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>ISBN</title>
    <link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='styles.css') }}">
</head>
<body>
    <h1>Registro</h1>
    <form method="POST" action="/record">
        <label for="user_name">Nombre del usuario:</label><br>
        <input type="text" id="user_name" name="user_name" required><br>
        <label for="user_name">Correo electrónico del usuario:</label><br>
        <input type="text" id="user_name" name="user_email"></input><br>
        <label for="task_id">ID de la tarea:</label><br>
        <input type="text" id="task_id" name="task_id"></input><br>
        <label for="task_name">Nombre de la tarea:</label><br>
        <input type="text" id="task_name" name="task_name" required></input><br>
        <label for="task_description">Descripción de la tarea:</label><br>
        <input type="text" id="task_description" name="task_description"></input><br>
        <button type="submit">Crear</button>
    </form>
</body>
</html>

**7.** En el directorio **static**, crear archivo **styles.css**:

In [ ]:
body {
    font-family: Arial, sans-serif;
    margin: 500;
    padding: 0;
    background-color: #f4f4f4;
}

.container {
    width: 20%;
    margin: 0 auto;
    padding: 20px;
    background-color: #fff;
    border-radius: 5px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
}

h1 {
    text-align: center;
    margin-bottom: 20px;
}

label {
    font-weight: bold;
}

input[type="text"],
textarea {
    width: 100%;
    padding: 10px;
    margin-bottom: 10px;
    border: 1px solid #ccc;
    border-radius: 3px;
    box-sizing: border-box;
}

textarea {
    height: 100px;
}

button {
    display: block;
    width: 100%;
    padding: 10px;
    background-color: #007bff;
    color: #fff;
    border: none;
    border-radius: 3px;
    cursor: pointer;
}

button:hover {
    background-color: #0056b3;
}


**8.** En la raíz del proyecto (directorio **isbn-wa**), crear archivo **app.py**:

In [ ]:
from flask import Flask
from controllers.record_controller import *

app = Flask(__name__)

app.register_blueprint(record_api)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=80)

**9.** En la raíz del proyecto (directorio **isbn-wa**), crear archivo **requirements.txt**:

In [ ]:
Flask==2.3.3
requests==2.31.0

**10.** En la raíz del proyecto (directorio **isbn-wa**), crear archivo **Dockerfile**:

In [ ]:
FROM python

WORKDIR /app

COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

COPY . .

EXPOSE 80

CMD ["python", "app.py"]

**11.** En la raíz del proyecto (directorio **isbn-wa**), crear archivo **config.py**:

In [ ]:
import os

# Environment Variables (API Gateway)

ISBN_AG_API_URL = os.environ.get('ISBN_AG_API_URL')

**12.** Crear imagen Docker para el componente **isbn-wa**:

In [ ]:
docker build -t isbn-wa .

**13.** Ejecutar el componente **isbn-wa** en un contenedor Docker:

*Nota:* reemplazar la variable **IP_DOCKER_CONTAINER** por la IP del contenedor del componente API Gateway. Para ello ejecutar:

> docker ps -a

Copiar el **CONTAINER ID** del contenedor **isbn_ag** y ejecutar:

> docker inspect CONTAINER ID

**IP_DOCKER_CONTAINER** = NetworkSettings > Networks > **IPAddress**

In [ ]:
docker run -p 80:80 -e ISBN_AG_API_URL=http://IP_DOCKER_CONTAINER:4500 --name isbn-wa isbn-wa

**14.** Abrir la aplicación web en un navegador web: [http://localhost/](http://localhost).

**15.** Crear un registro mediante la interfaz gráfica.

**16.** Verificar la creación de la tarea en las bases de datos de los dos microservicios.

### Despliegue en la Nube

Tanto el componente **isbn-ag**, como el componente **isbn-wa**, pueden ser desplegados usando los servicios **Cloud Run** o **Google Kubernetes Engine (GKE)**.